In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [6]:
# create_pegprod()

In [7]:
# create_prescriptions()

In [8]:
# create_consultations()

In [9]:
# create_clinicals()

In [10]:
# create_tests()

In [11]:
# create_referrals()

In [12]:
# create_immunisations()

In [13]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, male gender, insomnia count) used in the logistic regression*

In [5]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [6]:
all_entries = get_all_entries()

In [8]:
pt_features = create_pt_features()

In [9]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [10]:
pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype)

In [11]:
pt_features = add_data_start_and_end_dates(all_entries,pt_features)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
len(pt_features[pt_features['isCase']==True]),len(pt_features[pt_features['isCase']==False])

(47480, 47020)

In [13]:
pt_features.head(3)

,patid,yob,pracid,male,index_date,isCase,final dementia medcode,data_start,data_end
0,57001,32,001,0,2006-12-05,True,19477.0,1997-03-07,2013-05-17
1,60001,24,001,0,NaT,False,NaN,1997-03-07,2013-05-20
2,149001,19,001,1,2004-07-02,True,7572.0,1997-03-07,2004-11-17


In [14]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [15]:
for window in sd.exposure_windows:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,req_yrs_post_index=sd.req_yrs_post_index,start_year=abs(window['start_year']))
    pt_features = delete_unmatched_cases_and_controls(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7  being matched


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/zurfarosa/demres/common/process_pt_features.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

10_to_5  being matched
8_to_3  being matched


## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [16]:
medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [17]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [22]:
# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,codelists.codelist_list_of_lists)
    pt_features = create_pdds(pt_features,prescriptions,window,druglists.psychotropic_list_of_lists)
    pt_features = get_consultation_count(pt_features,all_entries,window)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7 ...
insomnia
	Total insomnia events in all medcoded_events dataframe: 60413
	insomnia events in this window for our patients: 1101
stroke
	Total stroke events in all medcoded_events dataframe: 48913
	stroke events in this window for our patients: 441
intellectual_disability
	Total intellectual_disability events in all medcoded_events dataframe: 163
	intellectual_disability events in this window for our patients: 6
CHD_heart_failure_and_peripheral_vascular_disease
	Total CHD_heart_failure_and_peripheral_vascular_disease events in all medcoded_events dataframe: 147199
	CHD_heart_failure_and_peripheral_vascular_disease events in this window for our patients: 1282
hypertension
	Total hypertension events in all medcoded_events dataframe: 185315
	hypertension events in this window for our patients: 2108
diabetes
	Total diabetes events in all medcoded_events dataframe: 57533
	diabetes events in this window for our patients: 523
clin_sig_alcohol_use
	Total clin_sig_alcohol_use events i

In [25]:
pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',').sample(10)

,patid,yob,pracid,male,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,insomnia,insomnia_consultations,stroke,intellectual_disability,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,clin_sig_alcohol_use,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,current_smoker,chronic_pulmonary_disease,CKD,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,benzo_and_z_drugs<1096,benzo_and_z_drugs>1096,benzo_and_z_drugs_never_used,other_sedatives_100_pdds,fgas_100_pdds,sgas_100_pdds,sga_depots_100_pdds,fga_depots_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations
25979,7894322,14,322,1,2005-08-19,True,4693.0,1995-09-21,2010-05-17,49148,91,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.072130,1,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,64
16565,2412271,33,271,1,2008-05-07,False,NaN,1999-09-22,2012-03-05,73328,75,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0.0,0.000000,0,0,1,0.275446,0.000000,0.0,0.0,0.0,0.000000,81
28190,3128166,29,166,0,2010-02-02,True,1917.0,1996-11-19,2013-01-24,24795,81,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,0,1,0.000000,0.000000,0.0,0.0,0.0,1.282391,122
10317,27931199,41,199,1,2010-11-29,True,1350.0,2001-10-24,2013-04-18,30207,69,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,7.776685,1,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,91
12249,8813294,30,294,1,2009-06-22,False,NaN,2001-06-08,2013-03-20,92995,79,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.0,7.847700,1,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,130
5731,11349426,27,426,1,2010-05-25,False,NaN,2001-10-16,2012-08-22,40248,83,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,0.000000,0,0,1,0.000000,0.000000,0.0,0.0,0.0,0.000000,122
5386,1913132,14,132,1,2008-09-25,False,NaN,2000-09-14,2011-07-04,40531,94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,0,1,0.000000,0.000000,0.0,0.0,0.0,0.000000,26
13661,15591227,21,227,1,2005-06-17,False,NaN,1995-10-05,2007-09-24,70294,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,0,1,0.000000,0.092446,0.0,0.0,0.0,0.000000,114
10926,8703573,20,573,1,2009-07-02,True,4693.0,1997-12-31,2009-08-04,83005,89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,0,1,0.000000,0.000000,0.0,0.0,0.0,0.000000,64
10235,97024,15,24,0,2007-04-12,False,NaN,1997-10-31,2009-04-21,37562,92,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,19.825128,0,1,0,0.000000,0.000000,0.0,0.0,0.0,4.964777,94
